TODO:
- top 10 trending videos on Youtube using Selenium
- set up a recurring job on AWS Lambda to scrape every 30 minutes
- send the results as a CSV over email

Why we need 'if __name__ == "__main__"': \
when we import some code from this file in another file, this can avoid executing all codes in this file

When I need to do some more complicated query, x-path is recommended

In [1]:
import requests

In [2]:
YOUTUBE_URL = 'https://www.youtube.com/feed/trending'

In [3]:
response = requests.get(YOUTUBE_URL)

In [4]:
print('Status code:', response.status_code)

Status code: 200


In [5]:
print('output', response.text[:1000])

output <!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="en" system-icons typography typography-spacing darker-dark-theme><head><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta http-equiv="origin-trial" content="AlgsH67ctYlMR3JYAxEnGfvsoFo41AMqAg6wRHHXOpWAfD54ZCfTPBclxnFT4Gc8IUX4pq6Xfo0esDuOt+WH3wIAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2ODA2NTI3OTksImlzU3ViZG9tYWluIjp0cnVlfQ=="/><script nonce="rUv3DPOVE61Y970KXlmHZA">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};
window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20230104.01.00');</script><script nonce="rUv3DPOVE61Y970KXlmHZA">(function

In [6]:
from bs4 import BeautifulSoup

In [7]:
doc = BeautifulSoup(response.text, 'html.parser')

In [11]:
print(doc.title)

<title>Trending - YouTube</title>


In [10]:
video_divs = doc.find_all('div', 
                          class_='style-scope ytd-video-renderer')

In [12]:
print(len(video_divs))

0


request does not execute Javascript.\
we need to create a fake browser.

In [13]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

In [14]:
def get_driver():
    service = Service(executable_path="/Users/chenyifan/Downloads/chromedriver")
    driver = webdriver.Chrome()
    driver.get(YOUTUBE_URL)
    return driver

In [15]:
def get_videos(driver):
    VIDEO_TAG = 'ytd-video-renderer'
    video_tags = driver.find_elements('tag name', VIDEO_TAG)
    print('Get {} videos'.format(len(video_tags)))
    return video_tags

In [16]:
driver = get_driver()

In [18]:
# I have to wait until the webpage is downloaded
# import time
# sleep(20)
videos = get_videos(driver)

Get 99 videos


In [19]:
# I want title, url, thumbnail_url, channel, views, uploaded, description
video = videos[0]
video_title = video.find_element('id', 'video-title')
url = video_title.get_attribute('href')
thumb_nail_img = video.find_element('tag name', 'img')
thumb_nail_url = thumb_nail_img.get_attribute('src')
channel = video.find_element('tag name', 'ytd-channel-name').find_element('tag name', 'a')
desc = video.find_element('id', 'description-text')

In [20]:
print('title:', video_title.text)
print('video_url:', url)
print('thumb_nail_url:', thumb_nail_url)
print('channel name:', channel.text)
print('video description:', desc.text)

title: Quavo - WITHOUT YOU
video_url: https://www.youtube.com/watch?v=_uS8QRe2qfE
thumb_nail_url: https://i.ytimg.com/vi/_uS8QRe2qfE/hqdefault.jpg?sqp=-oaymwE2CPYBEIoBSFXyq4qpAygIARUAAIhCGAFwAcABBvABAfgB_g6AArgIigIMCAAQARhlIGUoZTAP&rs=AOn4CLCZyLldTrLS8nAU6qrQXQQtar5t1Q
channel name: Quavo Huncho
video description: ♾🚀 Listen to "WITHOUT YOU" here: https://soundcloud.com/quavoofficial/quavo-without-you?si=d6d0a58a5baa4c82a8526ea03ec3359f&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing...


In [21]:
def parse_video(video):
    video_title = video.find_element('id', 'video-title')
    url = video_title.get_attribute('href')
    thumb_nail_img = video.find_element('tag name', 'img')
    thumb_nail_url = thumb_nail_img.get_attribute('src')
    channel = video.find_element('tag name', 'ytd-channel-name').find_element('tag name', 'a')
    desc = video.find_element('id', 'description-text')
    
    return {
        'title': video_title.text,
        'video_url': url,
        'thumb_nail_url': thumb_nail_url,
        'channel name': channel.text,
        'video description': desc.text
    }

In [22]:
videos_data = [parse_video(video) for video in videos[:10]]
videos_data

[{'title': 'Quavo - WITHOUT YOU',
  'video_url': 'https://www.youtube.com/watch?v=_uS8QRe2qfE',
  'thumb_nail_url': 'https://i.ytimg.com/vi/_uS8QRe2qfE/hqdefault.jpg?sqp=-oaymwE2CPYBEIoBSFXyq4qpAygIARUAAIhCGAFwAcABBvABAfgB_g6AArgIigIMCAAQARhlIGUoZTAP&rs=AOn4CLCZyLldTrLS8nAU6qrQXQQtar5t1Q',
  'channel name': 'Quavo Huncho',
  'video description': '♾🚀 Listen to "WITHOUT YOU" here: https://soundcloud.com/quavoofficial/quavo-without-you?si=d6d0a58a5baa4c82a8526ea03ec3359f&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing...'},
 {'title': 'I Strapped A GoPro To A Fish!',
  'video_url': 'https://www.youtube.com/watch?v=NigrQ9UcJy4',
  'thumb_nail_url': 'https://i.ytimg.com/vi/NigrQ9UcJy4/hqdefault.jpg?sqp=-oaymwEcCPYBEIoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBJsFZKpOxJFQxwaMF09GJE9mDGPQ',
  'channel name': 'Beast Reacts',
  'video description': "SUBSCRIBE OR YOU'LL HAVE BAD LUCK CHECK OUT THESE CHANNELS OR ELSE GoPro https://www.youtube.com/watch?v=8_T5oSUP-Kc https://www.

I can't find img src, it's because I don't give selenium enough time to load the page

In [23]:
import pandas as pd

In [24]:
videos_df = pd.DataFrame(videos_data)
videos_df

,title,video_url,thumb_nail_url,channel name,video description
0,Quavo - WITHOUT YOU,https://www.youtube.com/watch?v=_uS8QRe2qfE,https://i.ytimg.com/vi/_uS8QRe2qfE/hqdefault.j...,Quavo Huncho,"♾🚀 Listen to ""WITHOUT YOU"" here: https://sound..."
1,I Strapped A GoPro To A Fish!,https://www.youtube.com/watch?v=NigrQ9UcJy4,https://i.ytimg.com/vi/NigrQ9UcJy4/hqdefault.j...,Beast Reacts,SUBSCRIBE OR YOU'LL HAVE BAD LUCK CHECK OUT TH...
2,I'M PRINCESS PROTECTION PROGRAM,https://www.youtube.com/watch?v=kEnh_5z4I7w,https://i.ytimg.com/vi/kEnh_5z4I7w/hqdefault.j...,Danny Gonzalez,edited by Jake Mayer https://www.instagram.com...
3,Renfield | Official Trailer,https://www.youtube.com/watch?v=6LmO6rmDW08,https://i.ytimg.com/vi/6LmO6rmDW08/hqdefault.j...,Universal Pictures,🩸LET’S EAT🩸Sink your teeth into the new traile...
4,Shocking Evidence Links Bryan Kohberger to Ida...,https://www.youtube.com/watch?v=4cGGxOQz5SM,https://i.ytimg.com/vi/4cGGxOQz5SM/hqdefault.j...,Law&Crime Network,The probable cause affidavit police used to ar...
5,Popcaan - We Caa Done Ft Drake (Official Video),https://www.youtube.com/watch?v=TjJZNdLSexU,None,Popcaan,#Popcaan #Drake #GIHE Listen to We Caa Done: h...
6,2 Ex's Meet Again After ONE Year Of Not Seeing...,https://www.youtube.com/watch?v=snfCUuJuMN0,https://i.ytimg.com/vi/snfCUuJuMN0/hqdefault.j...,imLilPerfect,NEW Videos Everyweek! Turn My Notifications ON...
7,Sean McDermott And Josh Allen On Damar Hamlin’...,https://www.youtube.com/watch?v=cu3UwAX7rqY,https://i.ytimg.com/vi/cu3UwAX7rqY/hqdefault.j...,Buffalo Bills,Buffalo Bills head coach Sean McDermott and qu...
8,REVELATION // Episode 6 Cinematic - VALORANT,https://www.youtube.com/watch?v=XfuMEmcz3k0,https://i.ytimg.com/vi/XfuMEmcz3k0/hqdefault.j...,VALORANT,"From darkness, revelation. Uncover the wonders..."
9,THE ROAST OF ZANE HIJAZI,https://www.youtube.com/watch?v=ExxkvCSHKdc,https://i.ytimg.com/vi/ExxkvCSHKdc/hqdefault.j...,Jason Nash,It’s the “Roast of Zane Hijazi” sponsored by @...
